In [1]:
import requests 
import json 
import pandas as pd
import numpy as np
from tqdm import tqdm



In [2]:
URL = "https://api.steampowered.com/ISteamApps/GetAppList/v2/?access_token=eyAidHlwIjogIkpXVCIsICJhbGciOiAiRWREU0EiIH0.eyAiaXNzIjogInI6MDAxOF8yNUMzMEJBMF8xNEY1QyIsICJzdWIiOiAiNzY1NjExOTg4NzQ0MjkzOTAiLCAiYXVkIjogWyAid2ViOnN0b3JlIiBdLCAiZXhwIjogMTczODU4ODQ1NiwgIm5iZiI6IDE3Mjk4NjEzNjYsICJpYXQiOiAxNzM4NTAxMzY2LCAianRpIjogIjAwMERfMjVDMzBCQTFfMDgzREQiLCAib2F0IjogMTczODUwMTM2NiwgInJ0X2V4cCI6IDE3NTY0OTE4NDIsICJwZXIiOiAwLCAiaXBfc3ViamVjdCI6ICI0OS4yMDUuODYuMTc5IiwgImlwX2NvbmZpcm1lciI6ICI0OS4yMDUuODYuMTc5IiB9.TbQ3rhuSsvVa2SbefL9U1sx3FIDWJPMqhEi2OQJ_BFSdkZDm-TWPjaADJ3DGHe07H0zyQ0idGhVFa-_H1sAeDg"

res = requests.get(URL)

data_dict = res.json()['applist']['apps']
data_dict_cl = {}

for appid in data_dict:
    data_dict_cl[appid["appid"]] = appid["name"]

len(data_dict_cl)

def get_gamename(id):
    return data_dict_cl[id]



In [3]:
appl_df = pd.DataFrame(data_dict_cl.items(),columns=["app_id","app_name"])
appl_df = appl_df[(appl_df["app_id"] != '') & (appl_df["app_name"] != '')].reset_index()

In [53]:
appl_df[appl_df['app_id'] == 2967880]

,index,app_id,app_name
316,351,2967880,Deeper Than Hell Demo


In [52]:
appl_df.app_id[316]

2967880

In [6]:
appl_df[appl_df["app_name"] == ""]

,index,app_id,app_name


In [7]:
appl_df

,index,app_id,app_name
0,9,216938,Pieterw test app76 ( 216938 )
1,10,660010,test2
2,11,660130,test3
3,38,2972260,Ghosts
4,39,2972270,Dead Detention (Rescribbled) Demo
...,...,...,...
168503,168538,613830,CHRONO TRIGGER
168504,168539,629520,Soundpad
168505,168540,814480,Mini World: Block Art
168506,168541,814540,Changed


In [36]:
det_url = "http://store.steampowered.com/api/appdetails?appids={}"
var_int = 20006441
info_res = requests.get(det_url.format(var_int))

det_json = info_res.json()[str(var_int)]["data"]['name']
det_json


KeyError: 'data'

In [187]:
def app_data_1(df):
    det_url = "http://store.steampowered.com/api/appdetails?appids={}"
    app_data_df = pd.DataFrame(columns=["App Name","Release Date","Developers","Publishers"])
    for i in range(1,len(df['app_id'])):
        resp = requests.get(det_url.format(df['app_id'][i]))
        resp_op = resp.json()[str(df['app_id'][i])]['success']
        if(resp_op == "true"):
            print(df["app_id"][i])
            app_data_df['App Name'][i] = resp.json()[str(df['app_id'][i])]['data']['name']
            app_data_df['Release Date'][i] = resp.json()[str(df['app_id'][i])]['data']['release_date']
            app_data_df['Developers'][i] = resp.json()[str(df['app_id'][i])]['data']['developers']
            app_data_df['Publishers'][i] = resp.json()[str(df['app_id'][i])]['data']['publishers']
    return app_data_df 
        
        

In [45]:
def app_data(df):
    det_url = "http://store.steampowered.com/api/appdetails?appids={}"
    app_data_df = []
    for i in tqdm(range(0,len(df['app_id']))):
        resp = requests.get(det_url.format(df['app_id'][i]))
        resp_op = resp.json()[str(df['app_id'][i])]['success']
        if(str(resp_op) == "True"):
            item_type = resp.json()[str(df['app_id'][i])]['data']['type']
            plat_code = (str(resp.json()[str(df['app_id'][i])]['data']['platforms']['windows']) + str(resp.json()[str(df['app_id'][i])]['data']['platforms']['mac']) + str(resp.json()[str(df['app_id'][i])]['data']['platforms']['linux']))
            #print(str(df["app_id"][i]) +" "+str(resp_op))
            #print("inside func")
            if(str(item_type) == "game"):
                n = resp.json()[str(df['app_id'][i])]['data']['name']
                rd = resp.json()[str(df['app_id'][i])]['data']['release_date']['date'] if ('release_date' in resp.json()[str(df['app_id'][i])]['data']) else "-"
                devs = resp.json()[str(df['app_id'][i])]['data']['developers'] if ('developers' in resp.json()[str(df['app_id'][i])]['data']) else "-"
                pubs = resp.json()[str(df['app_id'][i])]['data']['publishers'] if ('publishers' in resp.json()[str(df['app_id'][i])]['data']) else "-"
                plats = plat_code
                app_data_df.append([n, rd, devs, pubs,plat_code])            
    return app_data_df 

In [46]:
data_set = app_data(appl_df)

  0%|                                | 315/168508 [05:12<46:19:48,  1.01it/s]


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [47]:
data_set

[['Ghosts',
  '4 Jun, 2024',
  ['Jaroslav Micek', 'Yarcom Interactive'],
  ['Jaroslav Micek']],
 ['Beauty Jigsaw', '16 May, 2024', ['2D_猫'], ['2D_猫']],
 ['Herald of the Mists', 'Coming soon', ['FromNiki'], ['FromNiki']],
 ['HFH', '12 Jun, 2024', ['GameP Ltd'], ['GameP Ltd']],
 ['Devious Path', '20 Sep, 2024', ['AAD Games'], ['AAD Games']],
 ['Wizdom Academy Playtest', '19 Jun, 2024', '-', '-']]

In [32]:
x = pd.DataFrame(data_set,columns = ["Name","Release Date","Developers","Publishers","Platforms"])
x

,Name,Release Date,Developers,Publishers
0,Ghosts,"4 Jun, 2024","[Jaroslav Micek, Yarcom Interactive]",[Jaroslav Micek]
1,Beauty Jigsaw,"16 May, 2024",[2D_猫],[2D_猫]
2,Herald of the Mists,Coming soon,[FromNiki],[FromNiki]
3,HFH,"12 Jun, 2024",[GameP Ltd],[GameP Ltd]
4,Devious Path,"20 Sep, 2024",[AAD Games],[AAD Games]
5,Wizdom Academy Playtest,"19 Jun, 2024",-,-


In [240]:
def app_data_keys(df):
    det_url = "http://store.steampowered.com/api/appdetails?appids={}"
    app_data_keys = []
    for i in tqdm(range(0,10)):
        resp = requests.get(det_url.format(df['app_id'][i]))
        resp_op = resp.json()[str(df['app_id'][i])]['success']
        #print(str(df["app_id"][i]) +" "+str(resp_op))
        if(str(resp_op) == "True"):
            #print("inside func")
            app_data_keys.append([[str(df['app_id'][i])], list(resp.json()[str(df['app_id'][i])]['data'].keys())]) 
    return app_data_keys 

In [241]:
test_keys = app_data_keys(appl_df)

100%|███████████████████████████████████████████| 10/10 [00:09<00:00,  1.03it/s]


In [148]:
df = appl_df
resp = requests.get(det_url.format(df['app_id'][2]))
resp_op = resp.json()[str(df['app_id'][2])]['success']
resp_op

False

In [242]:
test_keys

[[['2256500'],
  ['type',
   'name',
   'steam_appid',
   'required_age',
   'is_free',
   'detailed_description',
   'about_the_game',
   'short_description',
   'fullgame',
   'supported_languages',
   'header_image',
   'capsule_image',
   'capsule_imagev5',
   'website',
   'pc_requirements',
   'mac_requirements',
   'linux_requirements',
   'developers',
   'publishers',
   'price_overview',
   'packages',
   'package_groups',
   'platforms',
   'categories',
   'genres',
   'screenshots',
   'movies',
   'release_date',
   'support_info',
   'background',
   'background_raw',
   'content_descriptors',
   'ratings']],
 [['2256520'],
  ['type',
   'name',
   'steam_appid',
   'required_age',
   'is_free',
   'detailed_description',
   'about_the_game',
   'short_description',
   'supported_languages',
   'header_image',
   'capsule_image',
   'capsule_imagev5',
   'website',
   'pc_requirements',
   'mac_requirements',
   'linux_requirements',
   'developers',
   'publishers',
   

In [243]:
pd.DataFrame(test_keys)

,0,1
0,[2256500],"[type, name, steam_appid, required_age, is_fre..."
1,[2256520],"[type, name, steam_appid, required_age, is_fre..."
2,[2256540],"[type, name, steam_appid, required_age, is_fre..."
3,[2256550],"[type, name, steam_appid, required_age, is_fre..."
4,[2256600],"[type, name, steam_appid, required_age, is_fre..."
5,[2256640],"[type, name, steam_appid, required_age, is_fre..."
6,[2256650],"[type, name, steam_appid, required_age, is_fre..."
